# Bienvenido a la Jupyter Notebook de ejecución de visualizaciones


**(!)WARNING(!)**
**Condición necesaria para que el código funcione correctamente:**
**El archivo csv de lectura debe estar escrito todo en minúsculas.**

In [1]:
# Librerías Nativas de Python
import pickle, os

# Librerías de terceros
import pandas as pd
from dotenv import load_dotenv
from sklearn.preprocessing import OneHotEncoder
# import chart_studio.plotly as py
# import chart_studio.tools as tls

# Librería RD
from tools.feed import data_info

load_dotenv()

pd.options.plotting.backend = "plotly"

# USER = os.environ["USER_CS"]
# TOKEN = os.environ["TOKEN_CS"]
# tls.set_credentials_file(username=USER, api_key=TOKEN)

In [2]:
# APP
from app.time_series import TimeSeries


procesamiento = False
if procesamiento:
    TimeSeries.main()


##  cargar datos en vez de generarlos ya que podriamos haberlo corrido anteriormente
else:
    with open("./app/shared_resources/dfts_melcogatev2_10_20_1.pickle", "rb") as file:
        df = pickle.load(file)

In [3]:
print(os.listdir("./app/shared_resources/"))

['dfts_melcogatev2_10_20_1.pickle', 'dfts_melcogatev2_10_20_2.pickle', 'dfts_melcogatev2_10_20_3.pickle', 'dfts_melcogatev2_10_20_4.pickle', 'f1.pickle']


In [4]:
with open("./app/shared_resources/dfts_melcogatev2_10_20_1.pickle", "rb") as file:
    df = pickle.load(file)

In [5]:
df.iloc[:,9:20].head(2)

,coordinates,date,description_user,favorite_count,friends_count,host.ip,http.method,http.request.body.bytes,http.request.mime_type,http.version,is_quoted
0,-,"Oct 17, 2023 @ 21:00:00.000",vengo de a lla de los 70 \nya tenía mis 20 y a...,0,"1,333",192.168.40.26,POST,54574,application/json,HTTP/1.1,false
1,-,"Oct 17, 2023 @ 21:00:00.000",Músico.\nEl Peronismo nace del Fascismo de Mus...,0,440,192.168.40.26,POST,55914,application/json,HTTP/1.1,false


# Feature Selection

#### Features para el análisis de contexto

Cuando se quiere identificar qué es lo que está acontenciendo en determinado período de tiempo.
<br />
<center><b>

```
[datetime, content, token, username, usuarios_mencionados, hashtags, date_str, time_str, category]

[datetime, content, len(usuario_mencionado), len(hastags), spark_emotion, spark_sentiment, category]
```

</b></center>
<br />

#### Features para análisis de las conversaciones

Cuando se quiere identificar el fuerte de las conversaciones alrededor de un perfil determinado, es necesario conocer las fuentes que participan.

<br />
<center><b>

```


[quote_count, reach, replay_count, retweet_count, rt_like_count, rt_quote_count, rt_reply_count, rt_retweet_count, rt_text]

[category, type, author,  source, description_user, [favorite_count, friends_count, user_followers], [is_quoted, is_reply, is_retweet]]
 
```

</b></center>
<br />

 

In [6]:
## TEST 1
features = ["datetime", "content", "token", "usuarios_mencionados", "hashtags", "username", "date_str", "time_str", "category"]

for feature in features:
    if feature not in df.columns:
        raise Exception(f"Faltan features en la data recibida:\nFeature faltante \"{feature}\"")

In [7]:
# Obtenemos features e invertimos el orden del dataset
dataset = df.loc[::-1, features]

In [8]:
dataset.head(10)

,datetime,content,token,usuarios_mencionados,hashtags,username,date_str,time_str,category
7431,2023-10-15 13:17:00,la gran pregunta si estos audios surgieron de ...,"[gran, pregunta, audios, surgieron, investigac...","[@marianschw93348, @eligiria, @edufeiok]",NaN,Maria Sil💙⭐⭐⭐,2023-10-15,13:17:00,Twitter
7430,2023-10-15 13:18:00,por qué los votantes trolls de pb no quieren a...,"[votantes, trolls, quieren, admitir, chanta, m...",[@robertojg1810],NaN,◇Carolina Reaper◇,2023-10-15,13:18:00,Twitter
7429,2023-10-15 13:18:00,julito cómo te va? bullrich se está hundiendo ...,"[julito, bullrich, hundiendo, solita, audios, ...","[@jdgoldestein, @patobullrich, @jmilei, @llibe...",NaN,Patricia Mollo,2023-10-15,13:18:00,Twitter
7428,2023-10-15 13:18:00,"los periodistas , no hablan de los audios de m...","[periodistas, hablan, audios, melconia]",[@peponila],NaN,Ofe,2023-10-15,13:18:00,Twitter
7427,2023-10-15 13:18:00,le hubiese convenido mas quedarse comiendo piz...,"[convenido, quedarse, comiendo, pizza, casa, p...",[@gonziver],NaN,The Boogeyman,2023-10-15,13:18:00,Twitter
7426,2023-10-15 13:18:00,"no puede ser tan hijo de mil vecinos , cómo va...","[hijo, mil, vecinos, vida, privada, nadie, hab...",[@jonatanviale],NaN,Bautista V,2023-10-15,13:18:00,Twitter
7425,2023-10-15 13:18:00,? a que se debe esa conclusión? a lanata le es...,"[debe, conclusion, lanata, exigiendo, hable, m...","[@fperisset, @tata68276236]",NaN,Petula ll,2023-10-15,13:18:00,Twitter
7424,2023-10-15 13:19:00,increíblemente este tuit terminó siendo el res...,"[increiblemente, tuit, termino, resumen, expli...",NaN,NaN,Mau Grande,2023-10-15,13:19:00,Twitter
7423,2023-10-15 13:19:00,yo lo veo como q el votante de larreta ya deci...,"[veo, votante, larreta, decidio, voto, jxc, es...",[@esquizonomics],NaN,odio el rkt,2023-10-15,13:19:00,Twitter
7422,2023-10-15 13:19:00,mirtha le dio una paliza monumental al viejo v...,"[mirtha, dio, paliza, monumental, viejo, verde...",[@amazonaarg],[#melcogate],Th3Crak47OMG...,2023-10-15,13:19:00,Twitter


**Obtención del dataset de procesamiento**

**IMPORTANTE**

**Cuando trabajamos con series temporales, las primeras filas son los datos más antiguos y las últimas filas son los datos más nuevos.**

**Cosa que si nos llegan datos nuevos, se los agregamos al final.**

In [9]:
## TEST 2
features = ["datetime", "content", "spark_emotion", "spark_sentiment"]
for feature in features:
    if feature not in df.columns:
        raise Exception(f"Faltan features en la data recibida:\nFeature faltante \"{feature}\"")
# pendiente: , len(usuario_mencionado), len(hastags)
# pendiente: token

In [10]:
# Seleccionamos features e invertimos el orden del dataset
dataset_main = df.loc[::-1, features]

dataset_main["spark_emotion"] = [elemento if elemento != '-' else None for elemento in dataset_main.spark_emotion.to_list()]
dataset_main["spark_sentiment"] = [elemento if elemento != '-' else None for elemento in dataset_main.spark_sentiment.to_list()]

dataset_main

,datetime,content,spark_emotion,spark_sentiment
7431,2023-10-15 13:17:00,la gran pregunta si estos audios surgieron de ...,None,None
7430,2023-10-15 13:18:00,por qué los votantes trolls de pb no quieren a...,None,None
7429,2023-10-15 13:18:00,julito cómo te va? bullrich se está hundiendo ...,None,None
7428,2023-10-15 13:18:00,"los periodistas , no hablan de los audios de m...",None,None
7427,2023-10-15 13:18:00,le hubiese convenido mas quedarse comiendo piz...,None,None
...,...,...,...,...
4,2023-10-18 10:18:00,publico y notorio la complicidad de bullrich c...,None,None
3,2023-10-18 10:22:00,en serio creés que muchos votos que fueron par...,None,None
2,2023-10-18 10:22:00,y ni hablar de la complicidad de bullrich con ...,None,None
1,2023-10-18 10:34:00,jajajajaja más para entre ríos??? más para mee...,None,None


In [11]:
data_info(dataset_main)

,columna,Nan,pct_nan,dtype,count,pct_reg,count_unique
0,datetime,0,0.00,datetime64[ns],6916,100.00,2685
1,content,0,0.00,object,6916,100.00,6916
2,spark_emotion,6900,99.77,object,16,0.23,5
3,spark_sentiment,6900,99.77,object,16,0.23,2


In [12]:
## implementar features, cantidad de hastags, cantidad de usuarios mencionados

# Visualizaciones

**Frecuencias de los Marcos Temporales**

T: minuto <br />
H: hora <br />
D: día <br />
W: semana <br />
M: mes <br />
 <br />
5T = 5 minutos <br />
2D = 2 días <br />

In [13]:
# Funciones muy útiles para discretizar variables

# category_emotion, label_emotion = pd.factorize(df['spark_emotion'])
# category_sentiment, label_sentiment = pd.factorize(df['spark_sentiment'])

In [14]:
mask = dataset_main.spark_sentiment.notna()

# f: feature, m: muestras y n: dimensión de las features
f1 = dataset_main.loc[mask, :]
## Incorporar features generadas por dataset


# Crear una instancia de OneHotEncoder
encoder_i = OneHotEncoder(sparse_output=False)
encoder_ii = OneHotEncoder(sparse_output=False)

# Ajustar y transformar los datos
category_sentiment_encoded = encoder_i.fit_transform(f1[['spark_sentiment']])
category_emotion_encoded = encoder_ii.fit_transform(f1[['spark_emotion']])

# Convertir a DataFrame y unirlo al DataFrame original
sentiment_encoded = pd.DataFrame(category_sentiment_encoded, columns=encoder_i.get_feature_names_out(['spark_sentiment']), index=f1.index)
emotion_encoded = pd.DataFrame(category_emotion_encoded, columns=encoder_ii.get_feature_names_out(['spark_emotion']), index=f1.index)

f1 = pd.concat([f1, sentiment_encoded, emotion_encoded], axis=1)


# Establecer 'datetime' como índice
f1 = f1.set_index('datetime')
# f1 = f1.resample('60T').asfreq()
# Listado de las columnas que no son numéricas
non_numeric_column = data_info(f1).loc[(data_info(f1)["dtype"] != float).values].columna.values

f1_categorical = f1[non_numeric_column]
f1_numerical = f1.drop(non_numeric_column, axis=1)

### Resampleo de la muestra

In [15]:
freq = '4H'


    
    
f1_categorical = f1_categorical.resample(freq).agg(lambda x: list(x))
f1_numerical = f1_numerical.resample(freq).mean()

f1_resampled = pd.concat([f1_categorical, f1_numerical], axis=1)

f1_resampled.head(5)

,content,spark_emotion,spark_sentiment,spark_sentiment_NEGATIVO,spark_sentiment_NEUTRO,spark_emotion_DISGUSTO,spark_emotion_ENOJO,spark_emotion_FELICIDAD,spark_emotion_OTROS,spark_emotion_TRISTEZA
datetime,,,,,,,,,,
2023-10-16 04:00:00,[mientras no gaste en cosas innecesarias como ...,[ENOJO],[NEGATIVO],1.000000,0.000000,0.0,1.000000,0.0,0.000000,0.000000
2023-10-16 08:00:00,[],[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-10-16 12:00:00,"[melconian , un 4 de copas al lado de milei . ...","[ENOJO, DISGUSTO]","[NEGATIVO, NEGATIVO]",1.000000,0.000000,0.5,0.500000,0.0,0.000000,0.000000
2023-10-16 16:00:00,[que alguien me explique porqué la gente que v...,"[ENOJO, FELICIDAD]","[NEUTRO, NEGATIVO]",0.500000,0.500000,0.0,0.500000,0.5,0.000000,0.000000
2023-10-16 20:00:00,[te dejaron afuera en la mezasa…? fue con melc...,"[OTROS, ENOJO, TRISTEZA, OTROS, OTROS, OTROS]","[NEGATIVO, NEGATIVO, NEGATIVO, NEUTRO, NEGATIV...",0.666667,0.333333,0.0,0.166667,0.0,0.666667,0.166667


In [18]:
f1_visual = f1_resampled.filter(regex='^spark_sentiment_').fillna(0.)



visual_kwargs = {
    "width": 1200,
    "height": 600,
    "template": "plotly_dark",
    "title": f"Linea de tiempo de evolución de los sentimiento | Marco Temporal: {freq}",
    # "labels": {"variable": "Emociones", "value": "Porcentaje de emociones", "label": "Emociones"},
    # "color": "label",
    # "color_discrete_map": colores
    }


figure = f1_visual.plot(kind="line", **visual_kwargs).update_layout(showlegend=True)

f1_visual

c:\Users\Agustin\Desktop\RD\Octubre\RD_APP\venv\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()


,spark_sentiment_NEGATIVO,spark_sentiment_NEUTRO
datetime,,
2023-10-16 04:00:00,1.000000,0.000000
2023-10-16 08:00:00,0.000000,0.000000
2023-10-16 12:00:00,1.000000,0.000000
2023-10-16 16:00:00,0.500000,0.500000
2023-10-16 20:00:00,0.666667,0.333333
2023-10-17 00:00:00,0.000000,0.000000
2023-10-17 04:00:00,0.000000,1.000000
2023-10-17 08:00:00,1.000000,0.000000
2023-10-17 12:00:00,0.000000,1.000000


In [23]:
figure.show()


AttributeError: 'Figure' object has no attribute 'save'